<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/gemini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#pip install -q -U google-generativeai

In [11]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
import pandas as pd

In [13]:
model = genai.GenerativeModel('gemini-pro')

In [14]:
def detect(sender, body):
    try:
        #specify model we are using
        model=genai.GenerativeModel('gemini-pro')
        #specify prompt
        prompt =f"Sender: {sender}\nSMS Body: {body}\nIs this message real or fake?"
        # apply prompt to model and store in response object
        response= model.generate_content(prompt)

        #check if response is real or fake
        if "real" in response.text.lower():
            return "Real"
        elif "fake" in response.text.lower():
            return "Fake"
        else:
            return "Undetermined"

    except Exception as error:
        print("Error occured:", str(error))

In [19]:
from google.colab import files
uploaded = files.upload()

Saving smishTankSet.xlsx to smishTankSet (1).xlsx


In [16]:
#process excel file
def process_excel(inputFile):
  #assign file to pandas dataframe
  df=pd.read_excel(inputFile, header=None)
  #rename columns
  df.columns = ['Sender', 'SMS Body']

  # to store results at each value
  authenticityResults = []

  # process each row, apply detect function and print results
  for index, row in df.iterrows():
      #apply detect function on each row
      authResult = detect(row['Sender'], row['SMS Body'])
      authenticityResults.append(authResult)

      #print status after each orw
      print(f"Processed input {index + 1}: Sender = {row['Sender']}, Result = {authResult}")

  #assign results to new column
  df['Authenticity'] = authenticityResults

  #write updated dataframe to new file
  df.to_excel("processed_" + inputFile, index=False)


In [20]:
#process file
pFile = next(iter(uploaded))
process_excel(pFile)

Processed input 1: Sender = 42003, Result = Fake
Processed input 2: Sender = +1 (872) 279-0672, Result = Fake
Processed input 3: Sender = +1 (806) 224-7886, Result = Fake
Processed input 4: Sender = nan, Result = Fake
Processed input 5: Sender = +1 (775) 537-4497, Result = Fake
Processed input 6: Sender = +1 (276) 318-3831, Result = Fake
Processed input 7: Sender = +1 (859) 398-4745, Result = Fake
Processed input 8: Sender = +1 (626) 628-7248, Result = Fake
Processed input 9: Sender = +1 (805) 419-0678, Result = Fake
Processed input 10: Sender = +1 (646) 699-9070, Result = Real
Processed input 11: Sender = (803) 307-…, Result = Fake
Processed input 12: Sender = +1 (628) 946-9349, Result = Fake
Processed input 13: Sender = 6797002222, Result = Fake
Error occured: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.
Processed input 14: Sender = 18722795385, 

In [21]:
#download modified file to local machine
files.download("processed_" + pFile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>